## Busca de Imóveis

Utilizar sites de corretoras para criar um dataset de imóveis de Curitiba

In [160]:
from bs4 import BeautifulSoup, Comment
import requests
import re
from time import sleep
import csv

SITE = "https://www.zapimoveis.com.br/venda/imoveis/pr+curitiba/?onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,-25.437238,-49.269973,&pagina={pagina}&tipo=Im%C3%B3vel%20usado&transacao=Venda"

In [157]:

imoveis = []

paginas = [1,2,3,4,5,6,7,8,9,10]

for p in paginas:
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })

    link = requests.get(SITE.format(pagina=p), headers=headers)
    result = re.sub(r'<!.*?->','', link.content.decode("utf-8"))
    result = result.replace('\n','')

    reddit1Content =BeautifulSoup(result,"html.parser")


    cards = reddit1Content.find_all('div', class_='simple-card__box')


    for card in cards:
        imovel = []

        local = card.find('h2').text
        imovel.append(local.replace(' ',''))

        preco =card.find('p').find('strong').text
        imovel.append(preco.replace('R$','').replace(' ','').replace('.',''))

        try:
            metragem = card.find("span", itemprop="floorSize").text
            imovel.append(metragem.replace('m²','').replace(' ',''))
        except:
            imovel.append('')

        try:
            quartos = card.find("span", itemprop="numberOfRooms").text
            imovel.append(quartos.replace(' ',''))
        except:
            imovel.append('')

        try:
            garagens = card.find("li", class_="feature__item text-small js-parking-spaces")
            garagens = garagens.find('span', class_="").text
            imovel.append(garagens.replace(' ',''))
        except:
            imovel.append('')

        try:
            banheiros = card.find("span", itemprop="numberOfBathroomsTotal").text
            imovel.append(banheiros.replace(' ',''))
        except:
            imovel.append('')


        imoveis.append(imovel)
    sleep(30)

In [165]:
with open('imoveis.csv', 'w',newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['local','preco','metragem','quartos','garagens','banheiros'])
    writer.writerows(imoveis)